# Order Parameters

In this notebook, we analyze the order parameters of the cells.
We will use the information available in the folder `dat_order_parameters` which has the information of the global order parameters during the simulations at each time step and for each seed.

We define different order parameters in order to visualize how is the structure:

- Nematic Order ($Q$): .

- Label 2: A looser condition is used — cells are considered part of the same cluster if their overlap is at least 0.30 times the maximum.